In [1]:
from app.dotenv import base_dir, data_dir
from app.model.model import  Model
import os
model = os.getenv('llm_model')
print('ollama model:',model )
pdf_path = f"{data_dir}/{os.getenv('pdf_file')}.pdf"
print('pdf_path:',pdf_path )

ollama model: aya-expanse:32b
pdf_path: /Users/yishai/Desktop/InsuranceHebrewRAG-Assistant/data/כתב שירות רפואה משלימה.pdf


In [2]:
from ollama import chat,embed
import os

stream = chat(
    model=model,
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)
  
# embeding
embed(model=model, input='The sky is blue because of rayleigh scattering')

In [11]:
import pdfplumber
import re
def get_word_position(text):
    word_position = {}
    start_word = {}
    word_num = 0
    for line_num,line in enumerate(text.split('\n')):
        start_word[word_num] =  line_num
        for word in line.split(' '):
            # word_position[word_num] = line_num
            word_num+=1
    return start_word#,word_position

def format_text(text):
    new_text1 = ''
    for line in text.split('\n'):
        line = split_english_text(line)
        new_text1+=line + '\n'

    new_text2 = ''

    for line in new_text1.split('\n'):
        if not  re.match(r'^[a-zA-Z]', line.strip()):
            line = ''.join(reversed(line.strip()))
        else:
            line = line.strip() 
        new_text2+=line + '\n'
    return new_text2

def split_english_text(line):
    english_pattern = r'[A-Za-z]+'  # Matches English text and email addresses
    english_text = ' '.join(re.findall(english_pattern, text))
    start = english_text.split(' ')[0]
    end = english_text.split(' ')[-1]
    return line.replace(start, f'\n {start}').replace(end, f'{end} \n')
# Open the PDF file



def read_heb_pdf(pdf_path,header_height=60,footer_height=800):
    full_text = ''
      
    with pdfplumber.open(pdf_path) as pdf:
        # Loop through all the pages in the PDF
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            
            # Extract text from the page
            text = page.extract_text()
            lines = {} # Store the line position of each word in the text
            
            line_start_words = get_word_position(text)# get the line position of each word in the text
            for i,word in enumerate(page.extract_words()):# Extract characters and their details (position, font size, etc.) - compare to each line

                if i in line_start_words.keys():
                    line_num = line_start_words[i]
                    lines[line_num] = word
                    lines[line_num]['page_num'] = page_num
                    del lines[line_num]['upright'],lines[line_num]['doctop']#,lines[i]['text'],lines[line_num]['direction']
            
            new_text = ''        
            for line_num,line in enumerate(text.split('\n')):
                if lines[line_num]['top']<header_height or lines[line_num]['bottom']>footer_height or lines[line_num]['direction']=='ttb': 
                    continue 
                # print(lines[line_num]['height'])    
                new_text+=line + '\n'
                    
            # print('new_text:',new_text)
            #     if word['text']
                #     print('word_metadata:',word_metadata)
                #     print('word:',word)
            
            full_text += format_text(new_text)# split the text into lines of english and hebrew and than format it
            # break
    return full_text
full_text = read_heb_pdf(pdf_path)
print('full_text:',full_text)

    
    



full_text: תמצית תנאי הביטוח - כתב שירות - רפואה משלימה (3603)
תמצית פרטי הפוליסה
שם הביטוח כתב שירות - רפואה משלימה
סוג הביטוח כתב שירות
תקופת הביטוח מתחדשת כל שנתיים. ראה סעיף 01 לכתב השירות
תיאור הביטוח שירותי רפואה משלימה ופיזיותרפיה
הפוליסה אינה מכסה את המבוטח
ראה סעיף 6 לכתב השירות
במקרים הבאים (חריגים בפוליסה)
אחרי כמה זמן מתחילת הביטוח
09 ימים. ראה סעיף 71.2 לכתב השירות
ניתן לתבוע ולקבל תגמול (אכשרה)1
קבלת השירות מנותן שירות שבהסכם -
סך של 74 ₪ בעבור כל טיפול ו/או התייעצות שיבוצעו במרפאת נותן השירות שבהסכם.
סך של 27 ₪ בעבור כל טיפול ו/או התייעצות שנותן שירות שבהסכם יבצע בבית המנוי.
השתתפות עצמית
קבלת השירות מנותן שירות אחר -
שיפוי בסך של %05 מעלות הטיפול ששולמה בפועל אך לא יותר מ-041 ₪ לכל טיפול.
האם קיימים כיסויים חופפים בביטוח
רוב הכיסויים קיימים גם בשירותי בריאות נוספים של קופות החולים.
המשלים של קופות החולים
מחיר הביטוח החודשי ללא הנחות
גיל
ותוספות (ב - ₪)
0 עד 02 22.8
מגיל 12 ומעלה 68.12
עלות הביטוח
הסכומים צמודים למדד המחירים לצרכן 48631 שפורסם ביום 3202.80.51.
ניתן לשלם 

In [98]:

lines


{0: {'x0': 40.387020400000004,
  'x1': 97.4636229,
  'top': 42.938977499999964,
  'bottom': 60.04147750000004,
  'height': 17.102500000000077,
  'width': 57.0766025},
 2: {'x0': 173.873,
  'x1': 263.79200000000003,
  'top': 802.6569999999999,
  'bottom': 810.757,
  'height': 8.100000000000023,
  'width': 89.91900000000004},
 11: {'x0': 506.6519,
  'x1': 515.9039,
  'top': 805.6605999999999,
  'bottom': 814.6605999999999,
  'height': 9.0,
  'width': 9.25200000000001},
 15: {'x0': 126.9322,
  'x1': 184.45750000000004,
  'top': 812.657,
  'bottom': 820.757,
  'height': 8.099999999999909,
  'width': 57.525300000000044},
 30: {'x0': 18.22,
  'x1': 25.892500000000002,
  'top': 738.4748999999999,
  'bottom': 780.2749,
  'height': 41.80000000000007,
  'width': 7.672500000000003},
 31: {'x0': 82.6684,
  'x1': 134.8684,
  'top': 101.08540000000005,
  'bottom': 119.08540000000005,
  'height': 18.0,
  'width': 52.2},
 41: {'x0': 251.8466,
  'x1': 288.006735,
  'top': 142.30589999999995,
  'bottom'

In [68]:
# Print the text from the current page
print(f"Text from page {page_num + 1}:\n")
print(full_text)
print("\n" + "-"*80 + "\n")
# # break

# for lin in full_text.split('\n'):
#     if re.match(r'^[a-zA-Z]', lin.strip()):
#         print(lin.strip())


Text from page 11:

בבייטטווחח וופפייננננססייםם

BarClal@clal-ins.co.il
כלל חברה לביטוח בע"מ | חטיבת הבריאות |
1 | מתוך 11
www.clal.co.il
מען למשלוח דואר: ת.ד. 07073 תל אביב 2096316 | שירות לקוחות 4545* 0048836-30 |
20-1116-0923
תמצית תנאי הביטוח - כתב שירות - רפואה משלימה (3603)
תמצית פרטי הפוליסה
שם הביטוח כתב שירות - רפואה משלימה
סוג הביטוח כתב שירות
תקופת הביטוח מתחדשת כל שנתיים. ראה סעיף 01 לכתב השירות
תיאור הביטוח שירותי רפואה משלימה ופיזיותרפיה
הפוליסה אינה מכסה את המבוטח
ראה סעיף 6 לכתב השירות
במקרים הבאים (חריגים בפוליסה)
אחרי כמה זמן מתחילת הביטוח
09 ימים. ראה סעיף 71.2 לכתב השירות
ניתן לתבוע ולקבל תגמול (אכשרה)1
קבלת השירות מנותן שירות שבהסכם -
סך של 74 ₪ בעבור כל טיפול ו/או התייעצות שיבוצעו במרפאת נותן השירות שבהסכם.
סך של 27 ₪ בעבור כל טיפול ו/או התייעצות שנותן שירות שבהסכם יבצע בבית המנוי.
השתתפות עצמית
קבלת השירות מנותן שירות אחר -
שיפוי בסך של %05 מעלות הטיפול ששולמה בפועל אך לא יותר מ-041 ₪ לכל טיפול.
האם קיימים כיסויים חופפים בביטוח
רוב הכיסויים קיימים גם בשירותי בריא

In [37]:
import re

# Sample text
text = "BarClal@clal-ins.co.il | תואירבה תביטח | מ\"עב חוטיבל הרבח ללכ123"

# Regular expression to extract Hebrew and English parts
hebrew_pattern = r'[\u0590-\u05FF]+(?:[\u0590-\u05FF\s]*)'  # Matches Hebrew text


# Extract Hebrew and English parts
hebrew_parts = re.findall(hebrew_pattern, text)


# Join the parts into strings if needed
hebrew_text = ' '.join(hebrew_parts)

# Output the results
print("English part:", english_text)
print("Hebrew part:", hebrew_text)


English part: BarClal clal ins co il
Hebrew part: תואירבה תביטח  מ עב חוטיבל הרבח ללכ




BarClal@clal-ins.co.il

321כלל חברה לביטוח בע"מ | חטיבת הבריאות |
